In [1]:
param_x_api_key = '2BeBXg68o3pSd6GQvdyS5erm1aSyJUdKxZcgwsrEukJHa1mYjko5'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_part = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != 0:
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_part[input_.loc[i, 'JOIN_PNBI']] = {'vehicle_page': vehicle_page,
                                                 'crawler': 0,
                                                 'dict_vehicle': {},
                                                 'vehicle': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                data = '{"getAutoCarePartApplications":{"brandCode":"' + input_.loc[i, 'Brand_Id'] + '","partNumber":"' + input_.loc[i, 'Part_Number']  + '","includeResults":true,"includeMatchingBaseVehicles":true,"perPage":100,"page":' + str(j+1) + '}}'

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.opticatonline.com/autocare/v1/services/Catalog.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                for result in json_data['results']:
                    make = result['matchingBaseVehicles'][0]['makeName'].strip()
                    model = result['matchingBaseVehicles'][0]['modelName'].strip()
                    year = result['matchingBaseVehicles'][0]['year']

                    if make not in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle']:
                        dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make] = {model: [year]}
                    else:
                        if model not in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make]:
                            dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model] = [year]
                        else:
                            dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model].append(year)

                    dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model] = sorted(list(set(dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model])))

                dict_part[input_.loc[i, 'JOIN_PNBI']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_part[input_.loc[i, 'JOIN_PNBI']]['vehicle_page'] == dict_part[input_.loc[i, 'JOIN_PNBI']]['crawler']:
                    list_vehicle = []
                    for make in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle']:
                        for model in dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make]:
                            year_min = min(dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model])
                            year_max = max(dict_part[input_.loc[i, 'JOIN_PNBI']]['dict_vehicle'][make][model])

                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    list_vehicle.sort()

                    dict_part[input_.loc[i, 'JOIN_PNBI']]['vehicle'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'JOIN_PNBI': input_.loc[i, 'JOIN_PNBI'],
                                             'Vehicle': dict_part[input_.loc[i, 'JOIN_PNBI']]['vehicle']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[i, 'Part_Number'],
                                     'Brand_Id': input_.loc[i, 'Brand_Id'],
                                     'JOIN_PNBI': input_.loc[i, 'JOIN_PNBI']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'JOIN_PNBI'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_PNBI'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number', 'Brand_Id'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2050

[ok] - 1003-1028;DKHD
[尝试次数：1] - [剩余数量：2030] - [当前时间：13:32:51]

[ok] - 1003-1027;DKHD
[尝试次数：1] - [剩余数量：2029] - [当前时间：13:32:52]

[ok] - 1003-1029;DKHD
[尝试次数：1] - [剩余数量：2028] - [当前时间：13:32:52]

[ok] - 1003-1026;DKHD
[尝试次数：3] - [剩余数量：2027] - [当前时间：13:32:52]

[ok] - 1003-1025;DKHD
[尝试次数：5] - [剩余数量：2026] - [当前时间：13:32:53]

[ok] - 1003-1029;DKHD
[尝试次数：5] - [剩余数量：2025] - [当前时间：13:32:55]

[ok] - 1003-1031;DKHD
[尝试次数：1] - [剩余数量：2024] - [当前时间：13:32:55]

[ok] - 1003-1027;DKHD
[尝试次数：13] - [剩余数量：2023] - [当前时间：13:32:55]

[ok] - 1003-1022;DKHD
[尝试次数：14] - [剩余数量：2022] - [当前时间：13:32:56]

[ok] - 1003-1024;DKHD
[尝试次数：16] - [剩余数量：2021] - [当前时间：13:32:56]

[ok] - 1003-1028;DKHD
[尝试次数：16] - [剩余数量：2020] - [当前时间：13:32:57]

[ok] - 1003-1027;DKHD
[尝试次数：15] - [剩余数量：2019] - [当前时间：13:32:57]

[ok] - 1003-1028;DKHD
[尝试次数：18] - [剩余数量：2018] - [当前时间：13:32:58]

[ok] - 1003-1031;DKHD
[尝试次数：5] - [剩余数量：2017] - [当前时间：13:32:59]

[ok] - 1003-1032;DKHD
[尝试次数：5] - [剩余数量：2016] - [当前时间：13:33:01]

[ok] - 1003-1031;DKHD
[尝

[ok] - 210-1183;DKHD
[尝试次数：2] - [剩余数量：1901] - [当前时间：13:33:40]

[ok] - 210-1184;DKHD
[尝试次数：1] - [剩余数量：1900] - [当前时间：13:33:40]

[ok] - 210-1175;DKHD
[尝试次数：3] - [剩余数量：1899] - [当前时间：13:33:41]

[ok] - 210-1182;DKHD
[尝试次数：1] - [剩余数量：1898] - [当前时间：13:33:41]

[ok] - 210-1190;DKHD
[尝试次数：1] - [剩余数量：1897] - [当前时间：13:33:41]

[ok] - 210-91001;DKHD
[尝试次数：1] - [剩余数量：1896] - [当前时间：13:33:42]

[ok] - 210-91001;DKHD
[尝试次数：1] - [剩余数量：1895] - [当前时间：13:33:42]

[ok] - 210-91001;DKHD
[尝试次数：1] - [剩余数量：1894] - [当前时间：13:33:42]

[ok] - 210-91002;DKHD
[尝试次数：1] - [剩余数量：1893] - [当前时间：13:33:43]

[ok] - 210-91001;DKHD
[尝试次数：1] - [剩余数量：1892] - [当前时间：13:33:43]

[ok] - 210-91002;DKHD
[尝试次数：1] - [剩余数量：1891] - [当前时间：13:33:44]

[ok] - 210-91002;DKHD
[尝试次数：1] - [剩余数量：1890] - [当前时间：13:33:44]

[ok] - 210-91002;DKHD
[尝试次数：1] - [剩余数量：1889] - [当前时间：13:33:44]

[ok] - 210-91002;DKHD
[尝试次数：1] - [剩余数量：1888] - [当前时间：13:33:45]

[ok] - 210-91002;DKHD
[尝试次数：1] - [剩余数量：1887] - [当前时间：13:33:45]

[ok] - 210-91002;DKHD
[尝试次数：2] - [剩余数量：1886] 

[ok] - 273-10013;DKHD
[尝试次数：6] - [剩余数量：1772] - [当前时间：13:34:26]

[ok] - 273-10044;DKHD
[尝试次数：1] - [剩余数量：1771] - [当前时间：13:34:27]

[ok] - 273-10047;DKHD
[尝试次数：1] - [剩余数量：1770] - [当前时间：13:34:27]

[ok] - 1003-1020;DKHD
[尝试次数：28] - [剩余数量：1769] - [当前时间：13:34:27]

[ok] - 273-10021;DKHD
[尝试次数：7] - [剩余数量：1768] - [当前时间：13:34:28]

[ok] - 273-10063;DKHD
[尝试次数：4] - [剩余数量：1767] - [当前时间：13:34:29]

[ok] - 273-10031;DKHD
[尝试次数：4] - [剩余数量：1766] - [当前时间：13:34:29]

[ok] - 273-10026;DKHD
[尝试次数：11] - [剩余数量：1765] - [当前时间：13:34:29]

[ok] - 273-10029;DKHD
[尝试次数：6] - [剩余数量：1764] - [当前时间：13:34:29]

[ok] - 273-10142;DKHD
[尝试次数：1] - [剩余数量：1763] - [当前时间：13:34:30]

[ok] - 273-10024;DKHD
[尝试次数：14] - [剩余数量：1762] - [当前时间：13:34:30]

[ok] - 273-10143;DKHD
[尝试次数：3] - [剩余数量：1761] - [当前时间：13:34:31]

[ok] - 273-10140;DKHD
[尝试次数：7] - [剩余数量：1760] - [当前时间：13:34:31]

[ok] - 273-10213;DKHD
[尝试次数：4] - [剩余数量：1759] - [当前时间：13:34:33]

[ok] - 273-10042;DKHD
[尝试次数：16] - [剩余数量：1758] - [当前时间：13:34:33]

[ok] - 273-10199;DKHD
[尝试次数：8] - [剩余

[ok] - 210-1037;DKHD
[尝试次数：1] - [剩余数量：1643] - [当前时间：13:35:10]

[ok] - 210-1036;DKHD
[尝试次数：2] - [剩余数量：1642] - [当前时间：13:35:10]

[ok] - 210-1037;DKHD
[尝试次数：4] - [剩余数量：1641] - [当前时间：13:35:11]

[ok] - 210-1038;DKHD
[尝试次数：3] - [剩余数量：1640] - [当前时间：13:35:11]

[ok] - 210-1035;DKHD
[尝试次数：4] - [剩余数量：1639] - [当前时间：13:35:12]

[ok] - 210-1033;DKHD
[尝试次数：9] - [剩余数量：1638] - [当前时间：13:35:12]

[ok] - 210-1039;DKHD
[尝试次数：2] - [剩余数量：1637] - [当前时间：13:35:12]

[ok] - 210-1040;DKHD
[尝试次数：3] - [剩余数量：1636] - [当前时间：13:35:12]

[ok] - 210-1042;DKHD
[尝试次数：2] - [剩余数量：1635] - [当前时间：13:35:12]

[ok] - 210-1041;DKHD
[尝试次数：5] - [剩余数量：1634] - [当前时间：13:35:12]

[ok] - 210-1045;DKHD
[尝试次数：1] - [剩余数量：1633] - [当前时间：13:35:13]

[ok] - 210-1043;DKHD
[尝试次数：1] - [剩余数量：1632] - [当前时间：13:35:13]

[ok] - 210-1045;DKHD
[尝试次数：1] - [剩余数量：1631] - [当前时间：13:35:13]

[ok] - 210-1044;DKHD
[尝试次数：3] - [剩余数量：1630] - [当前时间：13:35:13]

[ok] - 210-1046;DKHD
[尝试次数：1] - [剩余数量：1629] - [当前时间：13:35:14]

[ok] - 210-1046;DKHD
[尝试次数：3] - [剩余数量：1628] - [当前时间：13:

[ok] - 211-1115;DKHD
[尝试次数：5] - [剩余数量：1513] - [当前时间：13:35:59]

[ok] - 273-10346;DKHD
[尝试次数：16] - [剩余数量：1512] - [当前时间：13:35:59]

[ok] - 211-1056;DKHD
[尝试次数：2] - [剩余数量：1511] - [当前时间：13:36:00]

[ok] - 211-1116;DKHD
[尝试次数：4] - [剩余数量：1510] - [当前时间：13:36:00]

[ok] - 211-1115;DKHD
[尝试次数：3] - [剩余数量：1509] - [当前时间：13:36:00]

[ok] - 211-1120;DKHD
[尝试次数：3] - [剩余数量：1508] - [当前时间：13:36:00]

[ok] - 273-10133;DKHD
[尝试次数：26] - [剩余数量：1507] - [当前时间：13:36:01]

[ok] - 211-1120;DKHD
[尝试次数：1] - [剩余数量：1506] - [当前时间：13:36:02]

[ok] - 211-1120;DKHD
[尝试次数：5] - [剩余数量：1505] - [当前时间：13:36:02]

[ok] - 211-1127;DKHD
[尝试次数：5] - [剩余数量：1504] - [当前时间：13:36:02]

[ok] - 211-1120;DKHD
[尝试次数：10] - [剩余数量：1503] - [当前时间：13:36:03]

[ok] - 211-1126;DKHD
[尝试次数：1] - [剩余数量：1502] - [当前时间：13:36:03]

[ok] - 211-1128;DKHD
[尝试次数：1] - [剩余数量：1501] - [当前时间：13:36:03]

[ok] - 211-1115;DKHD
[尝试次数：3] - [剩余数量：1500] - [当前时间：13:36:03]

[ok] - 211-1130;DKHD
[尝试次数：1] - [剩余数量：1499] - [当前时间：13:36:03]

[ok] - 211-1130;DKHD
[尝试次数：1] - [剩余数量：1498] - [当前时

...

[ok] - 250-241188;DKHD
[尝试次数：1] - [剩余数量：602] - [当前时间：13:41:11]

[ok] - 250-241185;DKHD
[尝试次数：5] - [剩余数量：601] - [当前时间：13:41:11]

[ok] - 250-241192;DKHD
[尝试次数：1] - [剩余数量：600] - [当前时间：13:41:11]

[ok] - 250-241188;DKHD
[尝试次数：2] - [剩余数量：599] - [当前时间：13:41:11]

[ok] - 250-241193;DKHD
[尝试次数：2] - [剩余数量：598] - [当前时间：13:41:12]

[ok] - 250-241189;DKHD
[尝试次数：3] - [剩余数量：597] - [当前时间：13:41:12]

[ok] - 250-241191;DKHD
[尝试次数：2] - [剩余数量：596] - [当前时间：13:41:12]

[ok] - 250-24119;DKHD
[尝试次数：4] - [剩余数量：595] - [当前时间：13:41:12]

[ok] - 250-241196;DKHD
[尝试次数：3] - [剩余数量：594] - [当前时间：13:41:12]

[ok] - 250-241203;DKHD
[尝试次数：1] - [剩余数量：593] - [当前时间：13:41:12]

[ok] - 250-241200;DKHD
[尝试次数：2] - [剩余数量：592] - [当前时间：13:41:13]

[ok] - 250-241144;DKHD
[尝试次数：4] - [剩余数量：591] - [当前时间：13:41:13]

[ok] - 250-241199;DKHD
[尝试次数：4] - [剩余数量：590] - [当前时间：13:41:13]

[ok] - 250-24121;DKHD
[尝试次数：1] - [剩余数量：589] - [当前时间：13:41:14]

[ok] - 250-241205;DKHD
[尝试次数：4] - [剩余数量：588] - [当前时间：13:41:14]

[ok] - 250-241162;DKHD
[尝试次数：3] - [剩余数量：58

[ok] - 250-24151;DKHD
[尝试次数：1] - [剩余数量：472] - [当前时间：13:42:09]

[ok] - 250-24150;DKHD
[尝试次数：1] - [剩余数量：471] - [当前时间：13:42:09]

[ok] - 250-24152;DKHD
[尝试次数：1] - [剩余数量：470] - [当前时间：13:42:09]

[ok] - 250-241161;DKHD
[尝试次数：5] - [剩余数量：469] - [当前时间：13:42:09]

[ok] - 250-24154;DKHD
[尝试次数：1] - [剩余数量：468] - [当前时间：13:42:10]

[ok] - 250-24155;DKHD
[尝试次数：1] - [剩余数量：467] - [当前时间：13:42:10]

[ok] - 250-24154;DKHD
[尝试次数：3] - [剩余数量：466] - [当前时间：13:42:10]

[ok] - 250-24153;DKHD
[尝试次数：4] - [剩余数量：465] - [当前时间：13:42:11]

[ok] - 250-24156;DKHD
[尝试次数：3] - [剩余数量：464] - [当前时间：13:42:11]

[ok] - 250-24157;DKHD
[尝试次数：2] - [剩余数量：463] - [当前时间：13:42:11]

[ok] - 250-24160;DKHD
[尝试次数：2] - [剩余数量：462] - [当前时间：13:42:12]

[ok] - 250-24158;DKHD
[尝试次数：3] - [剩余数量：461] - [当前时间：13:42:12]

[ok] - 250-24159;DKHD
[尝试次数：3] - [剩余数量：460] - [当前时间：13:42:12]

[ok] - 250-24163;DKHD
[尝试次数：1] - [剩余数量：459] - [当前时间：13:42:13]

[ok] - 250-24169;DKHD
[尝试次数：1] - [剩余数量：458] - [当前时间：13:42:13]

[ok] - 250-24169;DKHD
[尝试次数：2] - [剩余数量：457] - [当前时间：13

[ok] - 250-24301;DKHD
[尝试次数：4] - [剩余数量：342] - [当前时间：13:42:59]

[ok] - 250-24301;DKHD
[尝试次数：10] - [剩余数量：341] - [当前时间：13:42:59]

[ok] - 250-24302;DKHD
[尝试次数：2] - [剩余数量：340] - [当前时间：13:42:59]

[ok] - 250-24302;DKHD
[尝试次数：2] - [剩余数量：339] - [当前时间：13:43:00]

[ok] - 250-24302;DKHD
[尝试次数：3] - [剩余数量：338] - [当前时间：13:43:00]

[ok] - 250-24302;DKHD
[尝试次数：2] - [剩余数量：337] - [当前时间：13:43:00]

[ok] - 250-24301;DKHD
[尝试次数：10] - [剩余数量：336] - [当前时间：13:43:00]

[ok] - 250-24302;DKHD
[尝试次数：4] - [剩余数量：335] - [当前时间：13:43:01]

[ok] - 250-24307;DKHD
[尝试次数：1] - [剩余数量：334] - [当前时间：13:43:01]

[ok] - 250-24305;DKHD
[尝试次数：1] - [剩余数量：333] - [当前时间：13:43:01]

[ok] - 250-24305;DKHD
[尝试次数：2] - [剩余数量：332] - [当前时间：13:43:01]

[ok] - 250-24314;DKHD
[尝试次数：1] - [剩余数量：331] - [当前时间：13:43:02]

[ok] - 250-24294;DKHD
[尝试次数：12] - [剩余数量：330] - [当前时间：13:43:02]

[ok] - 250-24321;DKHD
[尝试次数：2] - [剩余数量：329] - [当前时间：13:43:02]

[ok] - 250-24310;DKHD
[尝试次数：5] - [剩余数量：328] - [当前时间：13:43:03]

[ok] - 250-24324;DKHD
[尝试次数：2] - [剩余数量：327] - [当前时间：

[ok] - 250-24441;DKHD
[尝试次数：2] - [剩余数量：212] - [当前时间：13:43:43]

[ok] - 250-24443;DKHD
[尝试次数：2] - [剩余数量：211] - [当前时间：13:43:44]

[ok] - 250-24440;DKHD
[尝试次数：5] - [剩余数量：210] - [当前时间：13:43:44]

[ok] - 250-24433;DKHD
[尝试次数：13] - [剩余数量：209] - [当前时间：13:43:44]

[ok] - 250-24439;DKHD
[尝试次数：6] - [剩余数量：208] - [当前时间：13:43:44]

[ok] - 250-24449;DKHD
[尝试次数：1] - [剩余数量：207] - [当前时间：13:43:45]

[ok] - 250-24435;DKHD
[尝试次数：13] - [剩余数量：206] - [当前时间：13:43:46]

[ok] - 250-24446;DKHD
[尝试次数：4] - [剩余数量：205] - [当前时间：13:43:46]

[ok] - 250-24447;DKHD
[尝试次数：5] - [剩余数量：204] - [当前时间：13:43:46]

[ok] - 250-24453;DKHD
[尝试次数：1] - [剩余数量：203] - [当前时间：13:43:47]

[ok] - 250-24450;DKHD
[尝试次数：4] - [剩余数量：202] - [当前时间：13:43:48]

[ok] - 250-24452;DKHD
[尝试次数：5] - [剩余数量：201] - [当前时间：13:43:48]

[ok] - 250-24453;DKHD
[尝试次数：1] - [剩余数量：200] - [当前时间：13:43:49]

[ok] - 250-24448;DKHD
[尝试次数：5] - [剩余数量：199] - [当前时间：13:43:49]

[ok] - 250-24454;DKHD
[尝试次数：3] - [剩余数量：198] - [当前时间：13:43:49]

[ok] - 250-24455;DKHD
[尝试次数：3] - [剩余数量：197] - [当前时间：1

[ok] - 250-24618;DKHD
[尝试次数：6] - [剩余数量：80] - [当前时间：13:44:32]

[ok] - 250-24617;DKHD
[尝试次数：10] - [剩余数量：79] - [当前时间：13:44:32]

[ok] - 250-24625;DKHD
[尝试次数：1] - [剩余数量：78] - [当前时间：13:44:33]

[ok] - 250-24610;DKHD
[尝试次数：20] - [剩余数量：77] - [当前时间：13:44:34]

[ok] - 250-24621;DKHD
[尝试次数：7] - [剩余数量：76] - [当前时间：13:44:34]

[ok] - 250-24627;DKHD
[尝试次数：3] - [剩余数量：75] - [当前时间：13:44:34]

[ok] - 250-24611;DKHD
[尝试次数：6] - [剩余数量：74] - [当前时间：13:44:34]

[ok] - 250-24620;DKHD
[尝试次数：10] - [剩余数量：73] - [当前时间：13:44:34]

[ok] - 250-24626;DKHD
[尝试次数：5] - [剩余数量：72] - [当前时间：13:44:34]

[ok] - 250-24629;DKHD
[尝试次数：4] - [剩余数量：71] - [当前时间：13:44:36]

[ok] - 250-24630;DKHD
[尝试次数：3] - [剩余数量：70] - [当前时间：13:44:36]

[ok] - 250-24634;DKHD
[尝试次数：4] - [剩余数量：69] - [当前时间：13:44:36]

[ok] - 250-24623;DKHD
[尝试次数：11] - [剩余数量：68] - [当前时间：13:44:37]

[ok] - 250-24623;DKHD
[尝试次数：15] - [剩余数量：67] - [当前时间：13:44:37]

[ok] - 250-24633;DKHD
[尝试次数：5] - [剩余数量：66] - [当前时间：13:44:37]

[ok] - 250-24641;DKHD
[尝试次数：1] - [剩余数量：65] - [当前时间：13:44:38]

[ok